#### EX 6.6 RetrievalQA

1. RetrievalQA

- retriever 를 사용하면 vectorstore 외에도 db, cloud 등에서도 문서를 찾아올수 있다.
- retriever : 구조화 되지 않은 query를 해석하여, document들을 반환하는데 vector store보다 일반적인 것이다. retrieve의 문서저장기능은 필수가 아니다. 반환만 하면 된다. vertor store를 retriever의 backborn(뼈대)로 사용할수도 있지만, 그렇지 않은 유형의 Retriever도 존재한다.

- 현재는 알려진 내용이 없기에 어떤 옵션이 있고, prompt를 바꾸는 방법이나 어떤 prompt가 적합한지, 어떻게 retriever가 쓰이는지에 대한 정보가 없다.

Stuff documents 형식으로 작동한다. - 기본적으로 모든 document를 prompt에 채워(stuff)넣는 것을 의미한다.

- vectorstore를 Chroma 에서 FAISS로 변경해보자. 조금 더 좋은 결과를 출력해준다.
  - FAISS는 이전에 받은 cache data를 사용하지 않는다.

AI가 질의에 응답하는 상황을 보려면 LangSmith에 가입하자.

> 가입신청한 LanSmith에서 초대가 오면 API KEY를 .env에 기록한다.
> https://smith.langchain.com/o/7f2804ef-91e3-5433-9926-cdca10ab6b33/


In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

# separator를 사용하면 특정 문자열을 지정하여 문장을 분할할수 있다.
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,  # 최대 글자 개수
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

# .cache에 저장된 embedding문서가 있으면 cache값을 읽어오고, 없으면 cache를 새로 만든다.
cached_embaddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embaddings)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # 검색 알고리즘을 설정란 refine, map_reduce, map_rerank 등
    retriever=vectorstore.as_retriever(),
)

chain.run("Describe Victory Mansions.")

'Victory Mansions is a building where Winston Smith resides. It has glass doors at the entrance, through which Winston enters, but they are not very effective at keeping out the wind and dust. The hallway of the building has a strong smell of boiled cabbage and old rag mats. There is a large colored poster on one wall depicting the face of a man in his forties, with a black mustache and ruggedly handsome features. The building has seven flights of stairs, as the lift is often not working due to the electricity being cut off during daylight hours. On each landing, there is a poster with the caption "BIG BROTHER IS WATCHING YOU." Inside Winston\'s flat, there is a telescreen, an oblong metal plaque that cannot be completely shut off, constantly broadcasting a voice reading out figures related to the production of pig-iron.'

- chain 알고리즘
  - stuff : 검색을 통해 관련 문서를 얻어서 prompt에 입력 후, 그 문서들과 함께 LLM에게 '질문에 답하세요' 라는 요청을 보냄
  - mapreduce : 각 문서를 순회하면서 답변을 탐색하고, 평가 후 평균에 가까운 답변들을 기반으로 최종 응답을 반환
  - map_rerank : 각 문서를 순회하면서 답변을 추출하는 대신 문서에 기반하여 질문에 대답하고 답변에 대한 점수를 기록한다. 10개의 문서가 있다면 10개에 문서에 대한 답변의 점수를 매기고 나중에 제일 높은 점수를 받은 문서를 결과로 반환
  - refine : 첫번쨰 문서를 읽어서 답변 초안을 생성하고, 순차적으로 모든 문서를 읽을 때 까지 답변을 계속해서 업데이트 후 반환.
